In [1]:
from keras import layers, models
import glob
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.colors as colors
from matplotlib.patches import Polygon
from mpl_toolkits.basemap import Basemap
from matplotlib.colors import rgb2hex
from keras import regularizers
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [2]:
path = "C:\\Users\\Sam\\Documents\\cal poly\\3\\spring\\cs466\\project\\KDD_Group_Project\\datasets"

In [3]:
final = pd.read_csv(path + "\\fatal.csv")

In [4]:
coords = pd.read_csv(path + "\\city_coordinates.csv")

In [5]:
tmp = np.array([x.split(" ") for x in coords["Location "]])

In [6]:
newtmp = np.array([[" ".join(x[:-1]), x[-1]] for x in tmp])

In [7]:
coords["city"] = newtmp[:,0]
coords["state"] = newtmp[:,1]

In [8]:
finaldf = pd.merge(final, coords, how = "left", on = ["city", "state"])

In [9]:
popdens = {final.iloc[x]["state"]: final.iloc[x]["state_pop"] for x in range(len(final))}

In [10]:
counts = Counter(finaldf.state)
for i in counts:
    counts[i] = counts[i] / popdens[i]

In [19]:
from matplotlib.collections import PatchCollection

mapfile = "C:\\Users\\Sam\\Desktop\\Research - Crow White\\data\\USA_adm_shp"
colors = {}
ccc = []
statenames = []
indices = {}
fig = plt.figure(figsize = (20,15))
ax = fig.add_subplot(111)
m = Basemap(llcrnrlon=-119,llcrnrlat=22,urcrnrlon=-64,urcrnrlat=49,
        projection='lcc',lat_1=33,lat_2=45,lon_0=-95)
m.ax = ax
m.readshapefile(mapfile + '\\USA_adm1', name='states', drawbounds=True)

cmap = plt.cm.viridis
vmin = min(counts.values())
vmax = max(counts.values())
for shapedict in range(len(m.states_info)):
    statename = m.states_info[shapedict]['HASC_1'].split(".")[-1]
    count = counts[statename]
    #if m.states_info[shapedict]["NAME_1"] in colors:
    #    continue
    #else:
        #print(1 - np.sqrt((count - vmin)/(vmax-vmin)))
    colors[m.states_info[shapedict]["NAME_1"]] = cmap(np.sqrt((count - vmin)/(vmax-vmin)))[:3]
    statenames.append(m.states_info[shapedict]["NAME_1"])
    indices[shapedict] = m.states_info[shapedict]["NAME_1"]
    ccc.append(count)
    

m.drawcoastlines()
m.drawcountries()
m.drawmapboundary(fill_color='#99ffff')
m.fillcontinents(color='#cc9966', lake_color='#99ffff');
patches = []

for nshape, seg in enumerate(m.states):
    
    #if nshape in indices:
    color = rgb2hex(colors[indices[nshape]])
        #print(color)
    poly = Polygon(seg, facecolor = color, edgecolor = color)
    ax.add_patch(poly)
    patches.append(poly)
    

p = PatchCollection(patches, cmap=plt.cm.viridis)
p.set_array(np.array(ccc))
cb = fig.colorbar(p, ax=ax)

#plt.show()
plt.savefig("statedistplot.png")

# Neural network

In [12]:
clean = final[["manner_of_death", "armed", "age", "gender", "race", "state", "signs_of_mental_illness", "threat_level", "flee", "body_camera", "state_pop"]]

clean["signs_of_mental_illness"] = [1 if clean.iloc[x]["signs_of_mental_illness"] == True else 0 for x in range(len(clean))]
clean["body_camera"] = [1 if clean.iloc[x]["body_camera"] == True else 0 for x in range(len(clean))]

clean = clean.dropna()

np.random.seed(0)
clean = clean.sample(frac = 1)
labels = clean["race"]

C:\Users\Sam\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Sam\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [13]:
dumm = pd.get_dummies(clean[[x for x in clean.columns if x != "race"]])
labels = pd.get_dummies(labels)
input_size = len(dumm.columns)
alld = dumm.values
alllabels = labels.values
xtrain, ytrain = alld[:int(.8 * len(alld)),], alllabels[:int(.8 * len(alld)),]
xtest, ytest = alld[int(.8 * len(alld)):,], alllabels[int(.8* len(alld)):,]

In [14]:
newx = []
newy = []
for i in range(len(xtrain)):
    if np.argmax(ytrain[i]) != 5:
        newx.append(xtrain[i])
        newx.append(xtrain[i])
        newy.append(ytrain[i])
        newy.append(ytrain[i])
    else:
        newx.append(xtrain[i])
        newy.append(ytrain[i])

In [15]:
xtrain = np.array(newx)
ytrain = np.array(newy)

In [16]:
nn = models.Sequential()
nn.add(layers.Dense(8, input_shape = (input_size,), kernel_regularizer=regularizers.l2(.01), activation = "sigmoid"))
nn.add(layers.Dropout(.2))
nn.add(BatchNormalization())
nn.add(layers.Dense(8, activation = "sigmoid", kernel_regularizer=regularizers.l2(.01)))
nn.add(layers.Dense(6, activation = "softmax"))

nn.compile(
    optimizer="rmsprop",             # Improved backprop algorithm
    loss='categorical_crossentropy', # "Misprediction" measure
    metrics=['accuracy']             # Report CCE value as we train
)

In [17]:
nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 1136      
_________________________________________________________________
dropout_1 (Dropout)          (None, 8)                 0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8)                 32        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 54        
Total params: 1,294
Trainable params: 1,278
Non-trainable params: 16
_________________________________________________________________


In [18]:
hst = nn.fit(xtrain, ytrain, epochs = 20, batch_size = 64,
 validation_data = (xtest, ytest))

Train on 3593 samples, validate on 605 samples
Epoch 1/20
3593/3593 [==============================] - 1s 150us/step - loss: 2.2007 - acc: 0.0145 - val_loss: 2.0992 - val_acc: 0.0149
Epoch 2/20
3593/3593 [==============================] - 0s 27us/step - loss: 1.8675 - acc: 0.1222 - val_loss: 1.7943 - val_acc: 0.2579
Epoch 3/20
3593/3593 [==============================] - 0s 23us/step - loss: 1.6568 - acc: 0.3087 - val_loss: 1.6056 - val_acc: 0.2579
Epoch 4/20
3593/3593 [==============================] - 0s 22us/step - loss: 1.5217 - acc: 0.3437 - val_loss: 1.4702 - val_acc: 0.2579
Epoch 5/20
3593/3593 [==============================] - 0s 20us/step - loss: 1.4363 - acc: 0.3440 - val_loss: 1.4169 - val_acc: 0.2579
Epoch 6/20
3593/3593 [==============================] - 0s 21us/step - loss: 1.3846 - acc: 0.3421 - val_loss: 1.3023 - val_acc: 0.2579
Epoch 7/20
3593/3593 [==============================] - 0s 22us/step - loss: 1.3577 - acc: 0.3365 - val_loss: 1.2510 - val_acc: 0.5322
Epoch 8